
# 04 · Privacy Risk → ROI & Final Report

This notebook links compliance posture to **business impact** and builds a **leadership‑ready PDF**:

- Load privacy & compliance JSONs from Notebooks 01–03  
- Configure **program economics** and **safeguard bundle**  
- Compute **ROI** of safeguards (avoided losses + time‑to‑market)  
- Run **one‑way sensitivity** (probability of incident, effectiveness)  
- Auto‑generate **`reports/privacy_compliance_report.pdf`** with charts


In [ ]:

import os, json, pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
import sys

# Import project modules
repo_root = Path.cwd()
if (repo_root / "notebooks").exists():
    scripts_dir = repo_root / "scripts"
    visuals_dir = repo_root / "visuals"
else:
    repo_root = Path.cwd().parent
    scripts_dir = repo_root / "scripts"
    visuals_dir = repo_root / "visuals"
for p in (scripts_dir, visuals_dir):
    if str(p) not in sys.path:
        sys.path.append(str(p))

from scripts.roi_privacy import PrivacyScenario, SafeguardBundle, roi_summary, sensitivity
from scripts.report_builder import build_privacy_pdf, ReportPaths

DATA_DIR = repo_root / "data"
REPORTS = repo_root / "reports"
ASSETS = REPORTS / "assets"
for d in (DATA_DIR, REPORTS, ASSETS):
    d.mkdir(exist_ok=True, parents=True)

print("Environment ready.")
print("repo_root =", repo_root)



## Load privacy & compliance results

These were produced by notebooks 01–03. (The notebook will still run if some files are missing; values will default.)


In [ ]:

def _load_json(p):
    try:
        return json.loads(Path(p).read_text())
    except Exception:
        return {}

privacy_report = _load_json(DATA_DIR / "privacy_report.json")
compliance_report = _load_json(DATA_DIR / "privacy_compliance_report.json")

print("Loaded keys: privacy:", list(privacy_report.keys())[:6], "| compliance:", list(compliance_report.keys())[:6])



## Configure program economics (edit as needed)

These values are **illustrative defaults**. Adjust to your program/portfolio.


In [ ]:

ts = PrivacyScenario(
    prob_incident_per_year=0.08,          # baseline incident probability
    regulatory_fine_usd=2_500_000.0,      # potential fine
    reputation_cost_usd=1_000_000.0,      # reputational remediation
    expected_delay_months_if_incident=4.0,# months delayed if incident
    delay_cost_per_month_usd=500_000.0,   # cost per delay month
    program_monthly_value_usd=3_000_000.0,# value of time-to-market
    discount_rate_annual=0.10             # WACC/hurdle
)
ts



## Configure safeguard bundle (edit as needed)

Includes **CapEx**, **OpEx**, and relative **effectiveness** (risk and delay reduction).


In [ ]:

sg = SafeguardBundle(
    cost_deid_capex_usd=400_000.0,
    cost_ppa_capex_usd=250_000.0,
    cost_governance_capex_usd=150_000.0,
    cost_annual_opex_usd=300_000.0,
    risk_reduction_relative=0.6,   # reduces incident probability by 60%
    delay_reduction_relative=0.5,  # reduces expected delay by 50%
    horizon_years=3
)
sg



## ROI summary


In [ ]:

roi = roi_summary(ts, sg)

# Save for report builder
(DATA_DIR / "privacy_roi_summary.json").write_text(json.dumps(roi, indent=2))
roi



## One‑way sensitivity

Explore how ROI changes with **incident probability** and **safeguard effectiveness**.


In [ ]:

grid = {
    "ts.prob_incident_per_year": [0.02, 0.05, 0.08, 0.12, 0.15],
    "sg.risk_reduction_relative": [0.3, 0.5, 0.7, 0.9]
}
rows = sensitivity(ts, sg, grid)
(DATA_DIR / "privacy_sensitivity.json").write_text(json.dumps(rows, indent=2))
pd.DataFrame(rows).head()



## Build final PDF report

Compiles metrics & charts into `reports/privacy_compliance_report.pdf`.  
If `reportlab` is not installed, a Markdown fallback will be created instead.


In [ ]:

out_path = build_privacy_pdf(
    title="Privacy & Compliance Readout",
    paths=ReportPaths(
        privacy_json=DATA_DIR / "privacy_report.json",
        compliance_json=DATA_DIR / "privacy_compliance_report.json",
        roi_json=DATA_DIR / "privacy_roi_summary.json",
        sensitivity_json=DATA_DIR / "privacy_sensitivity.json",
    )
)
out_path



## How to present to leadership

- **Headline**: “Safeguards reduce expected losses and time-to-market risk; ROI = Net Benefit / Cost.”  
- **Slider ranges**: Show sensitivity bands (p(incident), effectiveness) to highlight robustness.  
- **Action**: Approve the safeguard bundle; schedule quarterly re‑assessment; attach this report to DUA approvals.
